In [ ]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.layers import (Dense, Dropout, Flatten, BatchNormalization,
                                     Conv2D, Conv2DTranspose, Reshape, LeakyReLU, Input)
from tensorflow.keras.models import Sequential, Model
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.metrics import Mean
from tensorflow.keras.losses import BinaryCrossentropy
from tensorflow.keras.layers import Layer
import pathlib
import matplotlib.pyplot as plt
import numpy as np
import time

# Set dataset path
root_path = pathlib.Path("animefacedataset/images")

# Data Augmentation: Introduce minor transformations for better generalization
data_augmentation = keras.Sequential([
    keras.layers.RandomFlip("horizontal"),
    keras.layers.RandomRotation(0.1),
    keras.layers.RandomZoom(0.1),
])

# Load dataset and apply data augmentation
batch_size = 32
data = keras.utils.image_dataset_from_directory(
    directory=root_path,
    label_mode=None,  # No labels for GAN
    batch_size=batch_size,
    image_size=(64, 64)
).map(lambda x: data_augmentation((x - 127.5) / 127.5))  # Normalize images to [-1, 1]

def plot_images(images, title):
    plt.figure(figsize=(6, 6))
    for i in range(9):
        ax = plt.subplot(3, 3, i + 1)
        plt.imshow((images[i] * 127.5 + 127.5).numpy().astype("uint8"))  # Convert back to [0, 255]
        plt.axis("off")
    plt.suptitle(title)
    plt.show()

sample_images = next(iter(data))
plot_images(sample_images, "Sample Anime Faces")

class SpectralNormalization(Layer):
    def __init__(self, layer):
        super(SpectralNormalization, self).__init__()
        self.layer = layer

    def call(self, inputs):
        weights = self.layer.kernel
        sigma = tf.linalg.norm(weights, ord=2)  # Compute spectral norm
        self.layer.kernel = weights / sigma  # Normalize weights
        return self.layer(inputs)

def build_generator(latent_dim):
    model = Sequential([
        Dense(4 * 4 * 256, input_shape=(latent_dim,), use_bias=False),
        Reshape((4, 4, 256)),
        BatchNormalization(),
        LeakyReLU(),

        Conv2DTranspose(128, (4, 4), strides=2, padding='same', use_bias=False),
        BatchNormalization(),
        LeakyReLU(),

        Conv2DTranspose(128, (4, 4), strides=2, padding='same', use_bias=False),
        BatchNormalization(),
        LeakyReLU(),

        Conv2DTranspose(64, (4, 4), strides=2, padding='same', use_bias=False),
        BatchNormalization(),
        LeakyReLU(),

        Conv2DTranspose(3, (4, 4), strides=2, padding='same', activation='tanh')
    ])
    return model

G_model = build_generator(latent_dim=100)

def build_discriminator():
    inputs = Input(shape=(64, 64, 3))
    x = SpectralNormalization(Conv2D(64, (3, 3), strides=2, padding='same'))(inputs)
    x = LeakyReLU(0.2)(x)
    x = Dropout(0.3)(x)

    x = SpectralNormalization(Conv2D(128, (3, 3), strides=2, padding='same'))(x)
    x = LeakyReLU(0.2)(x)
    x = Dropout(0.3)(x)

    x = SpectralNormalization(Conv2D(256, (3, 3), strides=2, padding='same'))(x)
    x = LeakyReLU(0.2)(x)
    x = Dropout(0.3)(x)

    x = Flatten()(x)
    outputs = Dense(1, activation='sigmoid')(x)

    return Model(inputs, outputs)

D_model = build_discriminator()

class GAN(tf.keras.Model):
    def __init__(self, generator, discriminator, latent_dim):
        super(GAN, self).__init__()
        self.generator = generator
        self.discriminator = discriminator
        self.latent_dim = latent_dim
        self.g_loss_metric = Mean(name="g_loss")
        self.d_loss_metric = Mean(name="d_loss")

    def compile(self, g_optimizer, d_optimizer, loss_fn):
        super(GAN, self).compile()
        self.g_optimizer = g_optimizer
        self.d_optimizer = d_optimizer
        self.loss_fn = loss_fn

    @property
    def metrics(self):
        return [self.g_loss_metric, self.d_loss_metric]

    def train_step(self, real_images):
        batch_size = tf.shape(real_images)[0]
        random_latent_vectors = tf.random.normal((batch_size, self.latent_dim))
        fake_images = self.generator(random_latent_vectors)

        combined_images = tf.concat([real_images, fake_images], axis=0)
        labels = tf.concat([tf.ones((batch_size, 1)), tf.zeros((batch_size, 1))], axis=0)
        labels += 0.05 * tf.random.uniform(tf.shape(labels))  # Label smoothing

        with tf.GradientTape() as d_tape:
            d_predictions = self.discriminator(combined_images)
            d_loss = self.loss_fn(labels, d_predictions)

        d_grads = d_tape.gradient(d_loss, self.discriminator.trainable_weights)
        self.d_optimizer.apply_gradients(zip(d_grads, self.discriminator.trainable_weights))

        misleading_labels = tf.ones((batch_size, 1))
        with tf.GradientTape() as g_tape:
            g_predictions = self.discriminator(self.generator(random_latent_vectors))
            g_loss = self.loss_fn(misleading_labels, g_predictions)

        g_grads = g_tape.gradient(g_loss, self.generator.trainable_weights)
        self.g_optimizer.apply_gradients(zip(g_grads, self.generator.trainable_weights))

        self.d_loss_metric.update_state(d_loss)
        self.g_loss_metric.update_state(g_loss)

        return {"d_loss": self.d_loss_metric.result(), "g_loss": self.g_loss_metric.result()}

gan = GAN(G_model, D_model, latent_dim=100)
gan.compile(g_optimizer=Adam(2e-4, beta_1=0.5), d_optimizer=Adam(1e-4, beta_1=0.5), loss_fn=BinaryCrossentropy())

epochs = 50
history = gan.fit(data, epochs=epochs)

noise = tf.random.normal([16, 100])
generated_images = G_model(noise, training=False)
plot_images(generated_images, "Generated Anime Faces")

import tensorflow_hub as hub

# Load the pretrained InceptionV3 model
inception_model = hub.load('https://tfhub.dev/google/imagenet/inception_v3/classification/5')

def calculate_inception_score(images):
    # Resize images to (299, 299) as required by InceptionV3
    images = tf.image.resize(images, (299, 299))
    images = (images + 1.0) / 2.0  # Scale to [0, 1]
    preds = inception_model(images)
    p_yx = tf.nn.softmax(preds)
    p_y = tf.reduce_mean(p_yx, axis=0)

    kl_div = tf.reduce_sum(p_yx * (tf.math.log(p_yx) - tf.math.log(p_y)), axis=1)
    return tf.exp(tf.reduce_mean(kl_div))

def calculate_fid(real_images, generated_images):
    real_images_resized = tf.image.resize(real_images, (299, 299))
    fake_images_resized = tf.image.resize(generated_images, (299, 299))

    real_features = inception_model(real_images_resized)
    fake_features = inception_model(fake_images_resized)

    real_mu, real_sigma = tf.nn.moments(real_features, axes=[0])
    fake_mu, fake_sigma = tf.nn.moments(fake_features, axes=[0])

    diff = real_mu - fake_mu
    fid = tf.reduce_sum(diff**2) + tf.trace(real_sigma + fake_sigma - 2 * tf.linalg.sqrtm(real_sigma @ fake_sigma))
    return fid

# Example: Calculate Inception Score and FID for generated images
sample_generated_images = G_model(tf.random.normal([32, 100]), training=False)
is_score = calculate_inception_score(sample_generated_images)
fid_score = calculate_fid(sample_images, sample_generated_images)

print(f"Inception Score: {is_score.numpy()}, FID: {fid_score.numpy()}")

G_model.save("generator_model.h5")
D_model.save("discriminator_model.h5")

loaded_G_model = keras.models.load_model("generator_model.h5")
loaded_D_model = keras.models.load_model("discriminator_model.h5")

from keras_tuner import RandomSearch

def build_gan_model(hp):
    latent_dim = hp.Int("latent_dim", min_value=50, max_value=200, step=50)
    g_lr = hp.Choice("g_lr", values=[1e-4, 2e-4, 5e-4])
    d_lr = hp.Choice("d_lr", values=[1e-5, 1e-4, 1e-3])

    G_model = build_generator(latent_dim)
    D_model = build_discriminator()

    gan = GAN(G_model, D_model, latent_dim)
    gan.compile(
        g_optimizer=Adam(g_lr, beta_1=0.5),
        d_optimizer=Adam(d_lr, beta_1=0.5),
        loss_fn=BinaryCrossentropy(),
    )
    return gan

tuner = RandomSearch(
    build_gan_model,
    objective="val_loss",
    max_trials=5,
    executions_per_trial=1,
    directory="gan_tuning",
    project_name="anime_gan"
)

tuner.search(data, epochs=5, validation_data=data.take(1))
best_model = tuner.get_best_models(1)[0]

checkpoint_cb = keras.callbacks.ModelCheckpoint(
    "best_model.h5", save_best_only=True
)
early_stopping_cb = keras.callbacks.EarlyStopping(
    patience=5, restore_best_weights=True
)

gan.fit(data, epochs=50, callbacks=[checkpoint_cb, early_stopping_cb])

def generate_and_plot_images(generator, num_images=16):
    noise = tf.random.normal([num_images, 100])
    generated_images = generator(noise, training=False)
    plot_images(generated_images, "Generated Images")

generate_and_plot_images(G_model)

def train_gan(gan, dataset, epochs=50):
    for epoch in range(epochs):
        print(f"Epoch {epoch + 1}/{epochs}")
        start = time.time()

        for real_images in dataset:
            gan.train_step(real_images)

        # Generate sample images after every epoch
        generate_and_plot_images(gan.generator, 9)
        print(f"Time taken for epoch {epoch + 1}: {time.time() - start:.2f} sec")

train_gan(gan, data, epochs=50)